# 1. Import the libraries, load dataset, print shape of data, data description

In [4]:
import sys

!{sys.executable} -m pip install contractions
import re, string, unicodedata                          # Import Regex, string and unicodedata.
import contractions                                     # Import contractions library.
from bs4 import BeautifulSoup                           # Import BeautifulSoup.

import numpy as np                                      # Import numpy.
import pandas as pd                                     # Import pandas.
import nltk                                             # Import Natural Language Tool-Kit.

nltk.download('stopwords')                              # Download Stopwords.
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords                       # Import stopwords.
from nltk.tokenize import word_tokenize, sent_tokenize  # Import Tokenizer.
from nltk.stem.wordnet import WordNetLemmatizer         # Import Lemmatizer.
from sklearn.metrics import accuracy_score

  Using cached contractions-0.0.58-py2.py3-none-any.whl (8.0 kB)
  Using cached textsearch-0.0.21-py2.py3-none-any.whl (7.5 kB)
  Using cached anyascii-0.3.0-py3-none-any.whl (284 kB)
  Using cached pyahocorasick-1.4.2.tar.gz (321 kB)
  Running setup.py clean for pyahocorasick
Failed to build pyahocorasick
    Running setup.py install for pyahocorasick: started
    Running setup.py install for pyahocorasick: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\BALASUN1.LEGAL\Anaconda3\envs\Datascience\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\BALASUN1.LEGAL\\AppData\\Local\\Temp\\pip-install-10qr9vfc\\pyahocorasick_b9a4067986d84a738d8b2ca9fca1a8a5\\setup.py'"'"'; __file__='"'"'C:\\Users\\BALASUN1.LEGAL\\AppData\\Local\\Temp\\pip-install-10qr9vfc\\pyahocorasick_b9a4067986d84a738d8b2ca9fca1a8a5\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\BALASUN1.LEGAL\AppData\Local\Temp\pip-wheel-rgemct11'
       cwd: C:\Users\BALASUN1.LEGAL\AppData\Local\Temp\pip-install-10qr9vfc\pyahocorasick_b9a4067986d84a738d8b2ca9fca1a8a5\
  Complete output (5 lines):
  running bdist_wheel
  running bui

ModuleNotFoundError: No module named 'contractions'

In [ ]:
pip install pyahocorasick